Day 2: Understanding GPU Memory and Launch Configuration
Goal: Learn about different types of GPU memory (global, shared, and local) and configure the kernel execution parameters.

Tasks:
Understand GPU Memory Types:

**Global Memory**: This is the largest and slowest memory on the GPU. It’s used to store data that needs to be accessed by multiple threads.
Shared Memory: This is faster memory shared by threads within the same block. It's used to optimize performance for threads within a block.
Local Memory: This is memory local to each thread, but it is slower than shared memory and used when registers are insufficient.
Take note: For the task today, you’ll mainly work with global and shared memory.

Write a Kernel to Use Shared Memory:

We will create a kernel that uses shared memory to speed up computation (a simple summation of elements in an array).
The goal is to use shared memory to store intermediate results that can be accessed quickly by threads within the same block.

In [3]:
import numpy as np
from numba import cuda, float32

@cuda.jit
def sum_kernel(arr, result):
    # Define shared memory within a block
    shared = cuda.shared.array(256, dtype=float32)

    # Get the thread index
    idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x

    # Load data into shared memory
    if idx < arr.size:
        shared[cuda.threadIdx.x] = arr[idx]
    else:
        shared[cuda.threadIdx.x] = 0
    cuda.syncthreads()  # Synchronize threads before computation

    # Perform a parallel reduction within the block
    stride = 1
    while stride < cuda.blockDim.x:
        if cuda.threadIdx.x % (2 * stride) == 0:
            shared[cuda.threadIdx.x] += shared[cuda.threadIdx.x + stride]
        stride *= 2
        cuda.syncthreads()

    # The first thread in the block writes the result to global memory
    if cuda.threadIdx.x == 0:
        result[cuda.blockIdx.x] = shared[0]

# Initialize data
N = 1000000
arr = np.ones(N, dtype=np.float32)
result = np.zeros((N // 256), dtype=np.float32)  # Result for each block

# Transfer data to device
d_arr = cuda.to_device(arr)
d_result = cuda.to_device(result)

# Define block and grid size
threads_per_block = 256
blocks_per_grid = (arr.size + threads_per_block - 1) // threads_per_block

# Launch the kernel
sum_kernel[blocks_per_grid, threads_per_block](d_arr, d_result)

# Copy the result back to host
d_result.copy_to_host(result)

# Compute the final sum on the CPU by adding the results of each block
total_sum = np.sum(result)
print("Total sum:", total_sum)


Total sum: 999936.0


---


1. Import Libraries
*   numpy: A library for numerical computations, used here to create and manipulate arrays.
*   numba.cuda: Provides functionality for GPU programming using Numba's CUDA support.
*   float32: This is the data type we use for the shared memory array in the kernel.


---

2. Define the GPU Kernel
*  @cuda.jit: This decorator compiles the sum_kernel function into a GPU kernel, making it ready for execution on the GPU.
*  cuda.shared.array(256, dtype=float32): Creates a shared memory array of size 256, where 256 is the number of threads per block. This memory is faster than global memory and is shared by all threads within a block.


---

3. Thread Indexing
* cuda.threadIdx.x: Index of the current thread within its block (0 to blockDim.x - 1).
* cuda.blockIdx.x: Index of the current block in the grid (0 to gridDim.x - 1).
* cuda.blockDim.x: The number of threads in a block.

The overall index idx is calculated as threadIdx + blockIdx * blockDim, giving each thread a unique index across all blocks.


---

4. Load Data into Shared Memory
* Loading Data: Each thread loads an element from the global memory array arr into the shared memory shared.
* Index Bound Check: We check if the thread index idx is within the array size to avoid accessing out-of-bounds elements.
* cuda.syncthreads(): Synchronizes all threads in the block to ensure that all threads have finished loading data before any computation starts.

---
5. Parallel Reduction
* Parallel Reduction: This technique is used to sum the elements in shared memory. We repeatedly sum pairs of elements (at a distance of stride), reducing the array size by half with each iteration. This continues until only one value remains.
  * cuda.threadIdx.x % (2 * stride) == 0: This condition ensures that only threads with indices that are divisible by 2 * stride participate in the reduction step.
  * shared[cuda.threadIdx.x] += shared[cuda.threadIdx.x + stride]: Threads sum pairs of elements in the shared memory.
  * stride *= 2: Double the stride after each iteration to sum larger gaps.
* Synchronization: cuda.syncthreads() ensures that all threads in the block finish their operations before moving to the next step.

---
6. Write Final Result
*  After the parallel reduction, the first thread in each block (cuda.threadIdx.x == 0) writes the reduced sum from the shared memory (shared[0]) to the global memory result.
---
7. Initialize Data on Host (CPU)
* arr: An array of size N = 1,000,000 initialized with ones.
* result: An array that will store the partial sums from each block. The size of result is N // 256, as each block will compute one partial sum.

---
8. Transfer Data to GPU
*  cuda.to_device(): Transfers the input data arr and result from the CPU (host) to the GPU (device) memory.

---

9. Launch the Kernel
* Launch Configuration:
    * threads_per_block = 256: Specifies the number of threads per block. Each block will have 256 threads.
    * blocks_per_grid: The number of blocks is calculated based on the size of arr. Since each block processes threads_per_block elements, we divide the total size of arr by threads_per_block and round up to ensure every element is processed.
* Kernel Launch: The kernel is launched with blocks_per_grid blocks and threads_per_block threads per block.

---
10. Copy Result back to Host
*  This copies the computed partial results from the device back to the host (CPU) memory.

---
11. Final Sum on CPU
* The CPU calculates the total sum by summing up the partial results from each block stored in result.
* np.sum(result): Sums all the partial results.
---
**Summary**

* Shared Memory: Used for faster access within the block. We load data into shared memory and perform a parallel reduction to compute the sum more efficiently.
* Parallel Reduction: This technique allows threads in a block to collaborate and reduce the array size progressively, speeding up the summation process.
* Block and Grid Configuration: The kernel is launched with a grid of blocks, where each block processes a portion of the data using threads.
---